In [ ]:
#import packages 

In [ ]:
#system packages

import sys
import warnings
import os 
import traceback #obs? 
if not sys.warnoptions:
    warnings.filterwarnings("once")  

In [ ]:
#base packages:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats, integrate, optimize
import math
import datetime
from datetime import date, timedelta

In [ ]:
#optional packages

from statsmodels.tsa.base.datetools import dates_from_str
from IPython.display import display
pd.options.display.max_columns = 50  #allow DF.head to show all columns in notebook
from see import see
from tabulate import tabulate 

In [ ]:
#packages for the econometrics / models
import statsmodels.api as sm
from statsmodels.tsa.vector_ar import vecm
from statsmodels.tsa.stattools import adfuller, zivot_andrews
from arch.unitroot import DFGLS, ADF, KPSS, PhillipsPerron, ZivotAndrews
from arch.unitroot.cointegration import engle_granger, phillips_ouliaris
import statsmodels.formula.api as smf  #VAR package contained within 
import statsmodels.tsa.api as smt

#import the functionality for detecting mathematical errors (E.G. types of linear algebra issues etc.)
from statsmodels.tools.sm_exceptions import ValueWarning
from arch.utility.exceptions import (
    InfeasibleTestException,
    InvalidLengthWarning,
    invalid_length_doc)
warnings.filterwarnings("once", category = ValueWarning)

In [ ]:
#with ADF results: to be updated to VECM outcome (cointegration) eventually?
sorted_alldata_df_final_adf_results = pd.read_pickle("./sorted_alldata_df_final_adf_results.pkl")

In [ ]:
class color:
   #a simplified way to easily call different style elements for displaying outputs
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   GREEN = '\033[92m'
   HEADER = '\033[95m' 
   OKBLUE = '\033[94m'
   WARNING = '\033[93m'
   FAIL = '\033[91m'
   END = '\033[0m'

In [ ]:
#Original VECM base: CDS/PECDS

gvkey_selection = 1408
gvkey_frame = sorted_alldata_df_final_adf_results.loc[sorted_alldata_df_final_adf_results.gvkey==gvkey_selection]
#coint_rels = 0

print("VECM MODEL OF GVKEY: " + str(gvkey_selection))

cds = gvkey_frame['market_cds_spread']
pecds = gvkey_frame['pecds']
cboe_vix = gvkey_frame['cboe_vix']
vecm_data_base = pd.DataFrame(data=(cds, pecds)).T

vecm_data_base = vecm_data_base[(vecm_data_base.index <= np.percentile(vecm_data_base.index, 75))]

vecm_model_base = vecm.select_order(vecm_data_base, maxlags=5)
optimal_lag_length_base = vecm_model_base.bic
    
vecm_cointeg_rank_base = vecm.select_coint_rank(vecm_data_base, 
                                                det_order =1, 
                                                k_ar_diff = optimal_lag_length_base, 
                                                method='trace', 
                                                signif=0.05)

if vecm_cointeg_rank_base.rank >= 1:
    coint_rels_base = 1
else:
    print("insufficient number of cointegrating relationships for VECM model")
    
vecm_base = vecm.VECM(vecm_data_base, 
                      k_ar_diff=vecm_model_base.bic, 
                      coint_rank=coint_rels_base, 

                      deterministic='co')

#exog=cboe_vix

vecm_base_res = vecm_base.fit()
print(vecm_base_res.summary())




print('\n', vecm_base_res.test_granger_causality(caused=0).summary(), '\n', '--'*25, '\n', 
     vecm_base_res.test_granger_causality(caused=1).summary())

In [ ]:
cholesky_decomp = np.linalg.cholesky(vecm_base_res.sigma_u)
sig_1 = cholesky_decomp[0, 0]
sig_12 = cholesky_decomp[1, 0]
sig_2 = cholesky_decomp[1, 1]

x_1 = vecm_base_res.alpha[0,0]
x_2 = vecm_base_res.alpha[1,0]

gg_val = np.round(vecm_base_res.alpha[1,0] / vecm_base_res.alpha[1,0] - vecm_base_res.pvalues_alpha[0,0], 2)

HAS_1 = np.round(( x_2**2 * (sig_1**2 - sig_12**2 / sig_2**2) )  / ( x_2**2 * sig_1**2 - 2*x_1*x_2*sig_12 + x_1**2 * sig_2**2 ),4)
HAS_2 = np.round(( (x_2 * sig_1 - x_1 * sig_12 / sig_1)**2 ) / ( x_2**2 * sig_1**2 - 2*x_1*x_2*sig_12 + x_1**2 * sig_2**2 ),4)
HAS_mid = np.round((HAS_2 + HAS_1) / 2,4)

print(tabulate([['GG VALUE', gg_val],
       ['HAS LOWER', HAS_1],
       ['HAS MID', HAS_mid],
       ['HAS UPPER', HAS_2]],
       headers=[color.BOLD + 'PRICE DISCOVERY', 'VALUE' + color.END], tablefmt='pretty', numalign='center'),
     '\n', color.GREEN, "[ VALUES > 0.5 FAVOUR CDS |&| VALUES < 0.5 FAVOUR BONDS ]", color.END)

In [ ]:
## e.g. full length == 1485
## 75% trim = 114


In [ ]:
len(vecm_data_base)

In [ ]:
len(cboe_vix) 

In [1]:
1114 * 1.25

1392.5

In [2]:
np.floor(1114 * 1/0.75)

NameError: name 'np' is not defined

In [3]:
1114 * 1/0.75

1485.3333333333333

In [4]:
## need to convert to floor 
0.25 * 1485

371.25

In [ ]:
0.25 * len(vecm_data_base)

In [ ]:
vecm_base_res.predict(371)

In [ ]:
vecm_base_res.plot_forecast(steps=371)

In [ ]:
## TRY ## 

from sklearn.metrics import mean_squared_error

In [1]:
#TBC:
# further regression models and outputs with regressions on the firm variables!

#from statsmodels.iolib.summary2 import summary_col   #proably won't work as for panel reg
#https://stackoverflow.com/questions/54881902/generate-statistical-tables-in-python-and-export-to-excel

#from pystout import pystout
#!{sys.executable} -m pip install stargazer
#from stargazer.stargazer import Stargazer   #table/regression output package 



## cointegrating examples: (good) statsmodels
#https://arch.readthedocs.io/en/latest/unitroot/unitroot_cointegration_examples.html
#http://rizaudinsahlan.blogspot.com/2017/09/johansen-cointegration-test-with-eviews_93.html

#consider: moving data cleaning to seperate document? 

#see msvecm_1.02: breakdown of the VECM process explanation + links 

# APPENDIX (B)
* Correlation Analysis: VIX/BOND/CDS/STOCK CLOSE
* ## https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html

In [ ]:
## correlation analysis with VIX etc. 


In [ ]:
gvkey_selection = 1045 # 176404 #3226 #1440 #3226 #1078

gvkey_group_df = sorted_alldata_df_final_adf_results.loc[sorted_alldata_df_final_adf_results.gvkey==gvkey_selection]
print(" COMPANY == ", gvkey_group_df.company_name.iloc[0], '\n', "SECTOR ==  ", gvkey_group_df.sector.iloc[0])

In [ ]:
spearman_vix_pclose = stats.spearmanr(gvkey_group_df.cboe_vix, gvkey_group_df.price_close,
                                  nan_policy='omit')
spearman_vix_pclose

In [ ]:
spearman_pclose_vix = stats.spearmanr(gvkey_group_df.cboe_vix, gvkey_group_df.market_cds_spread,
                                  nan_policy='omit')
spearman_pclose_vix

In [ ]:
spearman_vix_tradingvol = stats.spearmanr(gvkey_group_df.cboe_vix, gvkey_group_df.trading_vol_daily,
                                  nan_policy='omit')
spearman_vix_tradingvol

In [ ]:
spearman_vix_pecds = stats.spearmanr(gvkey_group_df.cboe_vix, gvkey_group_df.pecds,
                                  nan_policy='omit')
spearman_vix_pecds

In [ ]:
spearman_pclose_tradingvol = stats.spearmanr(gvkey_group_df.price_close, gvkey_group_df.trading_vol_daily,
                                  nan_policy='omit')
print(spearman_pclose_tradingvol, '\n')
print("PVALUE ROUNDED: {}".format(np.round(spearman_pclose_tradingvol.pvalue, 4)))

In [ ]:
see(spearman_pclose_tradingvol)

In [ ]:
%%time 

warnings.filterwarnings("ignore") 

## initialise the dictionary
granger_causality_dict_4v = collections.defaultdict(list)

## initialise error counters by type
error_counter_dfgls = 0
#value_error_counter_dfgls = 0
error_counter_var = 0

## problem instances ##
unique_gct_4v_problem_instances = []

first_dif_counter = 0
second_dif_counter = 0
                              
## run the regression loop by GVKEY group:
for i, val in enumerate(sorted_alldata_df_final_adf_results.groupby('gvkey')):  
    gvkey = val[0]
    data = val[1]
    granger_causality_dict_4v['gvkey'].append(gvkey)  
    sector_string = data.sector
    granger_causality_dict_4v['sector'].append(sector_string.iloc[0]) 

    ## if all series are jointly stationary - apply the VAR model in-levels
    ## else take the first difference of each series and apply the VAR model in-differences
    try:
        
        cds = data.market_cds_spread 
        pecds = data.pecds
        vix = data.cboe_vix
        stock_close = data.price_close

        if stock_close.isna().any() == True:
            stock_close = stock_close.fillna(method='ffill')  

        #if np.allclose(cds, pecds, vix, stock_close) == False:
        #    print(len(cds), len(pecds), len(vix), len(stock_close))
        #    all_close_counter += 1
            #raise ValueError("Different Length Series")

        ## add try here 
        dfgls_cds = DFGLS(cds, trend='c', max_lags=10, method='BIC')
        dfgls_pecds = DFGLS(pecds, trend='c', max_lags=10, method='BIC')
        dfgls_vix = DFGLS(vix, trend='c', max_lags=10, method='BIC')
        dfgls_stock_close = DFGLS(stock_close, trend='c', max_lags=10, method='BIC')

        if ((dfgls_cds.stat < dfgls_cds.critical_values['5%']) and 
         (dfgls_pecds.stat < dfgls_pecds.critical_values['5%']) and 
         (dfgls_vix.stat < dfgls_vix.critical_values['5%']) and 
         (dfgls_stock_close.stat < dfgls_stock_close.critical_values['5%'])) == True:
            var_model_df_4v = pd.DataFrame(data=(cds, pecds, vix, stock_close)).T
            
            first_dif_counter += 1
            
            try:
                model = smt.VAR(var_model_df_4v)
                    
                max_lags_bic = model.select_order(maxlags=10).bic
                
                if max_lags_bic == 0:
                    max_lags_bic = 1

                res = model.fit(maxlags=max_lags_bic)

                ## test bonds on CDS causality ## 
                res_causality_bond = res.test_causality(causing=['pecds'], 
                    caused=['market_cds_spread'], kind='wald', signif=0.05)  

                ## test CDS on bonds causality ## 
                res_causality_cds = res.test_causality(causing=['market_cds_spread'], 
                    caused=['pecds'], kind='wald', signif=0.05) 

                ## test VIX / Stock Closing Price Jointly on CDS causality ## 
                res_causality_vixstock_cds = res.test_causality(causing=['cboe_vix','price_close'], 
                    caused=['market_cds_spread'], kind='wald', signif=0.05) 

                ## test VIX / Stock Closing Price Jointly on PECDS causality ##
                res_causality_vixstock_bond = res.test_causality(causing=['cboe_vix','price_close'], 
                    caused=['pecds'], kind='wald', signif=0.05)

                ## test VIX on closing price ##
                res_causality_vix_on_stock = res.test_causality(causing=['cboe_vix'], 
                    caused=['price_close'], kind='wald', signif=0.05) 

                ## calculate spearmanr coefficient of VIX / stock_close ## 
                vix_stock_close_spearman_corr = np.round(stats.spearmanr(vix, stock_close), 4)

                ## append items only after all GRANGER CAUSALITY TESTS ARE CONFIRMED
                granger_causality_dict_4v['vix_on_stock'].append(res_causality_vix_on_stock.conclusion)
                granger_causality_dict_4v['vix_stock_on_bond'].append(res_causality_vixstock_bond.conclusion)
                granger_causality_dict_4v['vix_stock_on_cds'].append(res_causality_vixstock_cds.conclusion)
                granger_causality_dict_4v['cds_on_bond'].append(res_causality_cds.conclusion)
                granger_causality_dict_4v['bond_on_cds'].append(res_causality_bond.conclusion)
                granger_causality_dict_4v['vix_stock_corr'].append(vix_stock_close_spearman_corr)
                
            except (ValueError, np.linalg.LinAlgError) as var_model_errors:
                error_counter_var +=1 
                granger_causality_dict_4v['bond_on_cds'].append(np.nan)
                granger_causality_dict_4v['cds_on_bond'].append(np.nan)
                granger_causality_dict_4v['vix_stock_on_cds'].append(np.nan)
                granger_causality_dict_4v['vix_stock_on_bond'].append(np.nan)    
                granger_causality_dict_4v['vix_on_stock'].append(np.nan)
                granger_causality_dict_4v['vix_stock_corr'].append(np.nan)
                
        else:
            
            

            cds = cds.pct_change().dropna()
            pecds = pecds.pct_change().dropna()
            vix = vix.pct_change().dropna()
            stock_close = stock_close.pct_change().dropna()

            dfgls_cds = DFGLS(cds, trend='c', max_lags=10, method='BIC')
            dfgls_pecds = DFGLS(pecds, trend='c', max_lags=10, method='BIC')
            dfgls_vix = DFGLS(vix, trend='c', max_lags=10, method='BIC')
            dfgls_stock_close = DFGLS(stock_close, trend='c', max_lags=10, method='BIC')

            if ((dfgls_cds.stat < dfgls_cds.critical_values['5%']) and 
             (dfgls_pecds.stat < dfgls_pecds.critical_values['5%']) and 
             (dfgls_vix.stat < dfgls_vix.critical_values['5%']) and 
             (dfgls_stock_close.stat < dfgls_stock_close.critical_values['5%'])) == True:
                var_model_df_4v = pd.DataFrame(data=(cds, pecds, vix, stock_close)).T
                
                try: 
                    model = smt.VAR(var_model_df_4v)
                    
                    max_lags_bic = model.select_order(maxlags=10).bic
                    
                    if max_lags_bic ==0:
                        max_lags_bic = 1
                    
                    res = model.fit(maxlags=max_lags_bic)
                    
                    ## test bonds on CDS causality ## 
                    res_causality_bond = res.test_causality(causing=['pecds'], 
                        caused=['market_cds_spread'], kind='wald', signif=0.05)  
                    
                    ## test CDS on bonds causality ## 
                    res_causality_cds = res.test_causality(causing=['market_cds_spread'], 
                        caused=['pecds'], kind='wald', signif=0.05) 
                    
                    ## test VIX / Stock Closing Price Jointly on CDS causality ## 
                    res_causality_vixstock_cds = res.test_causality(causing=['cboe_vix','price_close'], 
                        caused=['market_cds_spread'], kind='wald', signif=0.05) 
                    
                    ## test VIX / Stock Closing Price Jointly on PECDS causality ##
                    res_causality_vixstock_bond = res.test_causality(causing=['cboe_vix','price_close'], 
                        caused=['pecds'], kind='wald', signif=0.05)
                    
                    ## test VIX on closing price ##
                    res_causality_vix_on_stock = res.test_causality(causing=['cboe_vix'], 
                        caused=['price_close'], kind='wald', signif=0.05) 
                    
                    ## calculate spearmanr coefficient of VIX / stock_close ## 
                    vix_stock_close_spearman_corr = np.round(stats.spearmanr(vix, stock_close), 4)
                    
                    ## append items after all GCT have been run
                    granger_causality_dict_4v['vix_on_stock'].append(res_causality_vix_on_stock.conclusion)
                    granger_causality_dict_4v['vix_stock_on_bond'].append(res_causality_vixstock_bond.conclusion)
                    granger_causality_dict_4v['vix_stock_on_cds'].append(res_causality_vixstock_cds.conclusion)
                    granger_causality_dict_4v['cds_on_bond'].append(res_causality_cds.conclusion)
                    granger_causality_dict_4v['bond_on_cds'].append(res_causality_bond.conclusion)
                    granger_causality_dict_4v['vix_stock_corr'].append(vix_stock_close_spearman_corr[0])
                    
                except (ValueError, np.linalg.LinAlgError) as var_model_errors:
                    error_counter_var +=1 
                    granger_causality_dict_4v['bond_on_cds'].append(np.nan)
                    granger_causality_dict_4v['cds_on_bond'].append(np.nan)
                    granger_causality_dict_4v['vix_stock_on_cds'].append(np.nan)
                    granger_causality_dict_4v['vix_stock_on_bond'].append(np.nan)    
                    granger_causality_dict_4v['vix_on_stock'].append(np.nan)
                    granger_causality_dict_4v['vix_stock_corr'].append(np.nan)

            else:
                #print("1st Differences Not Jointly Stationary")
                second_dif_counter += 1
        
                granger_causality_dict_4v['bond_on_cds'].append("First_Diff_NS")
                granger_causality_dict_4v['cds_on_bond'].append("First_Diff_NS")
                granger_causality_dict_4v['vix_stock_on_cds'].append("First_Diff_NS")
                granger_causality_dict_4v['vix_stock_on_bond'].append("First_Diff_NS")   
                granger_causality_dict_4v['vix_on_stock'].append("First_Diff_NS")
                granger_causality_dict_4v['vix_stock_corr'].append("First_Diff_NS")

                
                ##### add in 2nd differences 
                ## add in error (1st differences not jointly stat)
            
    except (ValueError, np.linalg.LinAlgError, InfeasibleTestException) as dfgls_errors:
        error_counter_dfgls += 1
        #granger_causality_dict_4v['stock'].append("ERROR")
        granger_causality_dict_4v['bond_on_cds'].append(np.nan)
        granger_causality_dict_4v['cds_on_bond'].append(np.nan)
        granger_causality_dict_4v['vix_stock_on_cds'].append(np.nan)
        granger_causality_dict_4v['vix_stock_on_bond'].append(np.nan)    
        granger_causality_dict_4v['vix_on_stock'].append(np.nan)
        granger_causality_dict_4v['vix_stock_corr'].append(np.nan)


## construct the DF         
granger_causality_4v_df = pd.DataFrame.from_dict(granger_causality_dict_4v)        

print(" DFGLS ERRORS: {}".format(error_counter_dfgls),'\n', 
     "VAR MODEL ERRORS: {}".format(error_counter_var))